# Accuracy

Computation of the accuracy (according to insertion, deletion and $\mu$-fidelity of the spatial WCAM across different CNN backbones.

In [ ]:
!pip install xplique
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import pandas as pd
import os
from google.colab import drive
import json
import itertools

drive.mount('/content/gdrive')

In [2]:
from xplique.metrics import Deletion, Insertion, MuFidelity # evaluation methods
from spectral_sobol.tf_explainer import WaveletSobol
import keras.applications as app # models

In [3]:
# load the labels and the images
source_dir = 'folder/to/imagenet/explique/data'
target_dir = 'path/to/dest'

# number of iterations to compute the scores
reps = 1
batch_size = 128

# labels
labels = pd.read_csv(os.path.join(source_dir, 'labels.csv'))
y_wavelet = labels['label'].values
y = np.array([tf.keras.utils.to_categorical(label, 1000) for label in y_wavelet])

# images
images = [cv2.imread(os.path.join(source_dir, img_name))[..., ::-1] for img_name in labels['name'].values]


# helper that will update the file if it exists already
def update(target_dir, filename, data):
  """
  updates the file if it exists, creates and save otherwise
  """

  exists = os.path.exists(os.path.join(target_dir, filename))

  if exists:
    old = json.load(open(os.path.join(target_dir, filename)))
    
    # add the items
    for item in data.keys():
      old[item] = data[item]

    # save the file
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(old, f)

  else:
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(data, f)

In [ ]:
models = [
    app.VGG16(classifier_activation = 'softmax'), 
    app.ResNet50(classifier_activation = 'softmax'),
    app.MobileNet(classifier_activation = 'softmax'),
    app.EfficientNetV2B0(classifier_activation = 'softmax'),
]

# preprocessings for each model
preprocessings = [
    app.vgg16.preprocess_input,
    app.resnet.preprocess_input,
    app.mobilenet.preprocess_input,
    app.efficientnet.preprocess_input
]

names = ['VGG', 'ResNet', 'MobileNet', 'EfficientNet']

scores = {}

nb_design = 8
grid_size = 28

for name, model, preprocessing in zip(names, models, preprocessings):

  x = preprocessing(np.array(images, copy=True))

  #insertion = Insertion(model, x, y)
  #deletion = Deletion(model, x, y)
  fidelity = MuFidelity(model, x, y, batch_size, nb_samples=25)

  print(name)

  # explain the logits rather than the softmax
  model.layers[-1].activation = tf.keras.activations.softmax # 
  
  # set up the explainer and compute the explanations
  wavelet = WaveletSobol(model, grid_size = grid_size, nb_design = nb_design, batch_size = 512)
  explanations = wavelet(x,y_wavelet)

  # record the scores
  scores[name] = {
      #'insertion' : insertion(wavelet.spatial_cam),
      #'deletion'   : deletion(wavelet.spatial_cam)
      'mu_fidelity' : fidelity(wavelet.spatial_cam)
  }

  # save the results
  update(target_dir, "accuracy_mu_fidelity.json", scores)

VGG
ResNet
